In [12]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [13]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [14]:
reward_matrix = np.zeros([env.nS, env.nA], dtype=np.float)
value_index_matrix = np.empty([env.nS, env.nA], dtype=np.int)
for row in range(env.nS):
    for column in range(env.nA):
        reward_matrix[row][column] = env.P[row][column][0][2]
        value_index_matrix[row][column] = env.P[row][column][0][1]
def create_value_matrix(prev_value, discount_factor=1.0):
    func = lambda item:prev_value[item]
    vfunc = np.vectorize(func)
    value_matrix = reward_matrix + discount_factor * vfunc(value_index_matrix)
    return value_matrix
def create_policy_from_value(value_matrix):
    policy = np.zeros([env.nS, env.nA])
    for row in range(env.nS):
        max_column = np.argmax(value_matrix[row])
        policy[row][max_column] = 1.0
    return policy

In [15]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    iterate = 0
    while True:
        iterate = iterate + 1
        print('iterate is {}'.format(iterate))
        value_matrix = create_value_matrix(V)
        new_value = np.max(value_matrix, axis=1)
        if np.all(np.abs(new_value - V) < theta):
            break
        V = new_value
    
    policy = create_policy_from_value(value_matrix)
    
    return policy, V

In [16]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

iterate is 1
iterate is 2
iterate is 3
iterate is 4
Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [17]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)